In [2]:
import csv
from osgeo import gdal
from osgeo.gdalconst import GA_Update

In [2]:
def get_max_reflectance_value(image_path, band_index):
    # Open the raster dataset
    raster_ds = gdal.Open(image_path)

    if raster_ds is None:
        print("Error: Unable to open the raster dataset.")
        return None

    try:
        # Get the specified band
        band = raster_ds.GetRasterBand(band_index)

        # Read all pixel values in the band
        band_values = band.ReadAsArray()

        # Find the maximum reflectance value
        max_reflectance = band_values.max()

        return max_reflectance

    except Exception as e:
        print(f"Error: {e}")
        return None

    finally:
        # Close the raster dataset to release resources
        raster_ds = None

In [29]:

def create_tif_with_nonzero_max_reflectance(input_tif, output_tif):
    # Open the input TIFF file
    input_dataset = gdal.Open(input_tif, gdal.GA_ReadOnly)

    if input_dataset is None:
        print(f"Error: Could not open the file {input_tif}")
        return

    try:
        # Get the number of bands in the TIFF file
        num_bands = input_dataset.RasterCount

        non_zero_band_indices = []

        # Iterate through each band and copy non-zero bands to the new TIFF file
        for band_num in range(1, num_bands +1):
            max_reflectance = get_max_reflectance_value(input_tif, band_num)

            # Check if the band has non-zero maximum reflectance
            if max_reflectance != 0.0:
                non_zero_band_indices.append(band_num)

        # Create a new TIFF file with only non-zero bands
        if non_zero_band_indices:
            driver = gdal.GetDriverByName("GTiff")
            output_dataset = driver.Create(output_tif, input_dataset.RasterXSize,
                                           input_dataset.RasterYSize, len(non_zero_band_indices),
                                           gdal.GDT_Float32)  # Assuming the data type is Float32

            # Iterate through non-zero bands and copy data to the new TIFF file
            for new_band_index, original_band_index in enumerate(non_zero_band_indices, start=1):
                input_band = input_dataset.GetRasterBand(original_band_index)
                output_band = output_dataset.GetRasterBand(new_band_index)

                # Copy data from input band to output band
                data = input_band.ReadAsArray()
                output_band.WriteArray(data)

                # Set metadata and other properties if needed
                output_band.SetMetadata(input_band.GetMetadata())
                output_band.SetDescription(input_band.GetDescription())


        print(f"New TIFF file '{output_tif}' created successfully.")

    except Exception as e:
        print(f"Error: {e}")

    finally:
        # Close datasets to release resources
        input_dataset = None
        output_dataset = None


In [ ]:
input_tif_path = "raw_file.tif"
output_tif_path = "Filtered_non_zero.tif"
create_tif_with_nonzero_max_reflectance(input_tif_path, output_tif_path)

In [3]:
dataset = gdal.Open("corrected_raw_file.tif", gdal.GA_ReadOnly)
num_of_band = dataset.RasterCount

num_of_band

239